# Handling Prompt Complexity

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

#Balancing Details and Conciseness

In [4]:
from langchain_core.prompts import PromptTemplate

detailed_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""Please provide a comprehensive explanation of {topic}. Include its definition, 
    historical context, key components, practical applications, and any relevant examples. 
    Also, discuss any controversies or debates surrounding the topic, and mention potential 
    future developments or trends."""
)

concise_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Briefly explain {topic} and its main importance."
)

In [5]:
topic = "artificial intelligence"

print("Detailed response:")
print(llm.invoke(detailed_prompt.format(topic=topic)).content)

Detailed response:
Artificial Intelligence (AI) is a broad term that refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception. AI systems can be categorized into three main types: Narrow or Weak AI, General or Strong AI, and Superintelligence.

**Definition:**
Artificial Intelligence is the simulation of human intelligence in machines that are programmed to think, learn, and act like humans. AI systems are designed to perform tasks that would normally require human intelligence, such as:

* Reasoning and problem-solving
* Learning and adaptation
* Perception and understanding of the environment
* Natural language processing and understanding
* Decision-making and planning

**Historical Context:**
The concept of AI dates back to the Dartmouth Summer Research Project on Artificial Intelligence in 1956, where a group of computer scientists and mathematicians, including

In [6]:
print("\nConcise response:")
print(llm.invoke(concise_prompt.format(topic=topic)).content)


Concise response:
Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as:

1. Learning: AI systems can learn from data and improve their performance over time.
2. Reasoning: AI systems can make decisions and solve problems using logic and rules.
3. Perception: AI systems can interpret and understand data from sensors, images, and audio.

The main importance of AI lies in its potential to:

1. Automate tasks: AI can automate repetitive, mundane, and time-consuming tasks, freeing up human resources for more strategic and creative work.
2. Improve efficiency: AI can analyze large amounts of data quickly and accurately, making it an essential tool for industries such as healthcare, finance, and manufacturing.
3. Enhance decision-making: AI can provide data-driven insights and predictions, helping humans make better decisions in areas such as customer service, marketing, and logistics.
4. Drive in

#Analysis of Prompt Balance

In [7]:
analysis_prompt = PromptTemplate(
    input_variables=["detailed_response", "concise_response"],
    template="""Compare the following two responses on artificial intelligence:

Detailed response:
{detailed_response}

Concise response:
{concise_response}

Analyze the differences in terms of:
1. Information coverage
2. Clarity and focus
3. Potential use cases for each type of response

Then, suggest strategies for balancing detail and conciseness in prompts."""
)

In [8]:
detailed_response = llm.invoke(detailed_prompt.format(topic=topic)).content
concise_response = llm.invoke(concise_prompt.format(topic=topic)).content

analysis = llm.invoke(analysis_prompt.format(
    detailed_response=detailed_response,
    concise_response=concise_response
)).content

print(analysis)

**Differences:**

1. **Information coverage:** The detailed response covers a wider range of topics, including historical context, key components, practical applications, controversies, and future developments. The concise response, on the other hand, is more focused on the core concepts and potential uses of AI.
2. **Clarity and focus:** The concise response is more straightforward and easy to follow, with a clear and concise structure. The detailed response is more comprehensive, but may be overwhelming for readers who are new to the topic.
3. **Potential use cases:** The detailed response provides more examples and scenarios for AI applications, including healthcare, customer service, marketing, autonomous vehicles, and cybersecurity. The concise response mentions automation, efficiency, decision-making, and innovation, but does not provide specific examples.

**Strategies for balancing detail and conciseness:**

1. **Use a tiered approach:** Provide a brief overview of the topic, f

#Strategies for handling long context

##Chunking

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

long_text = """
Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines that can simulate human cognitive processes.
The field of AI has a rich history dating back to the 1950s, with key milestones such as the development of the first neural networks and expert systems.
AI encompasses a wide range of subfields, including machine learning, natural language processing, computer vision, and robotics.
Practical applications of AI include speech recognition, image classification, autonomous vehicles, and medical diagnosis.
AI has the potential to revolutionize many industries, from healthcare and finance to transportation and entertainment.
However, there are ongoing debates and controversies surrounding AI, such as concerns about job displacement, bias in algorithms, and the ethical implications of autonomous systems.
Looking ahead, the future of AI holds promise for advancements in areas like explainable AI, AI ethics, and human-AI collaboration. 
The intersection of AI with other technologies like blockchain, quantum computing, and biotechnology will likely shape the future of the field.
But as AI continues to evolve, it is essential to consider the societal impact and ethical implications of these technologies.
One of the key challenges for AI researchers and developers is to strike a balance between innovation and responsibility, ensuring that AI benefits society as 
a whole while minimizing potential risks.
If managed effectively, AI has the potential to transform our world in ways we can only begin to imagine.
Though the future of AI is uncertain, one thing is clear: the impact of artificial intelligence will be profound and far-reaching.
"""

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

# Split the text into chunks
chunks = text_splitter.split_text(long_text)

print(f"Number of chunks: {len(chunks)}")
print(f"First chunk: {chunks[0][:200]}...")

Number of chunks: 2
First chunk: Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines that can simulate human cognitive processes.
The field of AI has a rich history dating back to the...


##Summarization

In [ ]:
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

# Convert text chunks to Document objects
doc_chunks = [Document(chunk.page_content) for chunk in chunks]

# Load the summarization chain
chain = load_summarize_chain(llm, chain_type="map_reduce")

# Summarize the long text
summary_result = chain.invoke(doc_chunks)

print("Summary:")
print(summary_result['output_text'])

KeyboardInterrupt: 

##Iterative Processing

In [1]:
def iterative_analysis(text, steps):
    """
    Perform iterative analysis on a given text.
    
    Args:
    text (str): The text to analyze.
    steps (list): List of analysis steps to perform.
    
    Returns:
    str: The final analysis result.
    """
    result = text
    for step in steps:
        prompt = PromptTemplate(
            input_variables=["text"],
            template=f"Analyze the following text. {step}\n\nText: {{text}}\n\nAnalysis:"
        )
        result = llm.invoke(prompt.format(text=result)).content
    return result

In [ ]:
analysis_steps = [
    "Identify the main topics discussed.",
    "Summarize the key points for each topic.",
    "Provide a brief conclusion based on the analysis."
]

final_analysis = iterative_analysis(long_text, analysis_steps)
print("Final Analysis:")
print(final_analysis)

##Tips

In [ ]:
tips_prompt = """
Based on the examples and strategies we've explored for managing prompt length and complexity, 
provide a list of 5 practical tips for developers working with large language models. 
Each tip should be concise and actionable.
"""

tips = llm.invoke(tips_prompt).content
print(tips)